# 生成数据集

In [ ]:
import torch
import numpy as np

num_inputs = 2  # 是输入特征的数量，这里是 2。
num_examples = 1000  # 是样本的数量，这里是 1000。
true_w = [2, -3.4]
true_b = 4.2

features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

2. **设置参数**
   ```python
   num_inputs = 2
   num_examples = 1000
   true_w = [2, -3.4]
   true_b = 4.2
   ```
   - `num_inputs` 定义了输入特征的数量，这里为 2。
   - `num_examples` 定义了样本的数量，这里为 1000。
   - `true_w` 是真实的权重，值为 `[2, -3.4]`。
   - `true_b` 是真实的偏置，值为 `4.2`。

3. **生成特征数据**
   ```python
   features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
   ```
   - 使用 `numpy` 的 `np.random.normal` 函数生成一个形状为 `(1000, 2)` 的矩阵，其元素服从均值为 0，标准差为 1 的正态分布。
   - 将生成的 NumPy 数组转换为 PyTorch 的张量（`torch.tensor`），并指定数据类型为 `torch.float`。

4. **生成标签数据**
   ```python
   labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
   ```
   - 使用真实的权重和偏置计算每个样本的标签值。
   - `features[:, 0]` 获取所有样本的第一个特征，`features[:, 1]` 获取所有样本的第二个特征。
   - 计算公式为：`labels = 2 * features[:, 0] + (-3.4) * features[:, 1] + 4.2`。

5. **添加噪声**
   ```python
   labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
   ```
   - 使用 `numpy` 的 `np.random.normal` 函数生成一个与 `labels` 形状相同的噪声张量，其元素服从均值为 0，标准差为 0.01 的正态分布。
   - 将生成的噪声转换为 PyTorch 张量并添加到 `labels` 中，以模拟真实数据中的噪声。

### 总结

这段代码生成了一个简单的线性回归数据集，包括两个特征和一个带有噪声的标签。生成的数据集可以用于训练和测试线性回归模型。在这里：

- `features` 是输入特征矩阵，形状为 `(1000, 2)`。
- `labels` 是目标标签向量，形状为 `(1000,)`，其值是根据线性模型（加上噪声）计算得出的。

# 读取数据

In [ ]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

1. **导入模块**
   ```python
   import torch.utils.data as Data
   ```
   - 导入 PyTorch 的数据工具包，主要用于处理数据集和数据加载。

2. **设置批量大小**
   ```python
   batch_size = 10
   ```
   - 定义每个批量的数据量，这里设置为 10。

3. **创建数据集**
   ```python
   dataset = Data.TensorDataset(features, labels)
   ```
   - `Data.TensorDataset` 是一个简单的数据集封装器，用于将特征和标签组合在一起。
   - `features` 和 `labels` 分别是之前生成的特征张量和标签张量。
   - `dataset` 是一个包含输入特征和目标标签的数据集对象。

4. **创建数据加载器**
   ```python
   data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
   ```
   - `Data.DataLoader` 是 PyTorch 提供的一个数据加载器，能够以小批量的方式加载数据。
   - 参数：
     - `dataset`: 数据集对象。
     - `batch_size`: 每个批量的数据量，这里设置为 10。
     - `shuffle`: 是否在每个 epoch 开始时打乱数据，这里设置为 `True`，表示打乱数据。

### 使用

在训练过程中，可以使用 `data_iter` 来遍历数据集。每次迭代时，它将返回一个包含一小批特征和标签的元组。示例如下：

```python
for X, y in data_iter:
    print(X, y)
    break  # 只打印一个批量的数据
```

这样可以确保每次训练时都能以小批量的方式读取数据，并且每个 epoch 数据的顺序都是随机的，有助于提高模型的泛化能力。

In [ ]:
for X, y in data_iter:
    print(X, y)
    # print(data_iter)
    # break

# 定义模型

在上一节从零开始的实现中，我们需要定义模型参数，并使用它们一步步描述模型是怎样计算的。当模型结构变得更复杂时，这些步骤将变得更繁琐。其实，PyTorch提供了大量预定义的层，这使我们只需关注使用哪些层来构造模型。下面将介绍如何使用PyTorch更简洁地定义线性回归。

首先，导入torch.nn模块。实际上，“nn”是neural networks（神经网络）的缩写。顾名思义，该模块定义了大量神经网络的层。之前我们已经用过了autograd，而nn就是利用autograd来定义模型。nn的核心数据结构是Module，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。在实际使用中，最常见的做法是继承nn.Module，撰写自己的网络/层。一个nn.Module实例应该包含一些层以及返回输出的前向传播（forward）方法。下面先来看看如何用nn.Module实现一个线性回归模型。

==========================


当我们在 3.2节中实现线性回归时， 我们明确定义了模型参数变量，并编写了计算的代码，这样通过基本的线性代数运算得到输出。 但是，如果模型变得更加复杂，且当我们几乎每天都需要实现模型时，自然会想简化这个过程。 这种情况类似于为自己的博客从零开始编写网页。 做一两次是有益的，但如果每个新博客就需要工程师花一个月的时间重新开始编写网页，那并不高效。

对于标准深度学习模型，我们可以使用框架的预定义好的层。这使我们只需关注使用哪些层来构造模型，而不必关注层的实现细节。 我们首先定义一个模型变量net，它是一个Sequential类的实例。 Sequential类将多个层串联在一起。 当给定输入数据时，Sequential实例将数据传入到第一层， 然后将第一层的输出作为第二层的输入，以此类推。 在下面的例子中，我们的模型只包含一个层，因此实际上不需要Sequential。 但是由于以后几乎所有的模型都是多层的，在这里使用Sequential会让你熟悉“标准的流水线”。

回顾 图3.1.2中的单层网络架构， 这一单层被称为全连接层（fully-connected layer）， 因为它的每一个输入都通过矩阵-向量乘法得到它的每个输出。

在PyTorch中，全连接层在Linear类中定义。 值得注意的是，我们将两个参数传递到nn.Linear中。 第一个指定输入特征形状，即2，第二个指定输出特征形状，输出特征形状为单个标量，因此为1。

In [ ]:
import torch.nn as nn

net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
)

print(net)

线性层的数学公式可以用来解释其操作。具体来说，对于 `nn.Linear(num_inputs, 1)` 定义的线性层，其数学表达如下：

### 数学公式解释

#### 线性层操作

对于输入特征矩阵 \( X \) 和线性层的权重矩阵 \( W \) 和偏置项 \( b \)，线性层的输出 \( y \) 可以表示为：

$$ y = X \cdot W^T + b $$

其中：
- \( X \) 是输入特征矩阵，维度为 \((N, \text{num\_inputs})\)，其中 \(N\) 是样本数量，\(\text{num\_inputs}\) 是每个样本的特征数量。
- \( W \) 是权重矩阵，维度为 \((\text{num\_inputs}, 1)\)，其中 \(1\) 是输出特征的数量。
- \( b \) 是偏置项，通常是一个标量（或者在 PyTorch 中表示为一个维度为 \(1\) 的向量）。
- \( y \) 是输出特征矩阵，维度为 \((N, 1)\)。

#### 解释

1. **权重矩阵 \( W \)**:
   - 在 `nn.Linear(num_inputs, 1)` 中，权重矩阵 \( W \) 的维度是 \(\text{num\_inputs} \times 1\)。这是因为我们从 \(\text{num\_inputs}\) 个输入特征映射到 1 个输出特征。矩阵的转置 \( W^T \) 的维度是 \(1 \times \text{num\_inputs}\)，用于和输入矩阵 \( X \) 进行矩阵乘法。

2. **偏置 \( b \)**:
   - 偏置 \( b \) 是一个标量，表示输出特征的每个维度的偏移量。在实际计算中，偏置会被广播到输出的每个样本。

3. **前向传播计算**:
   - 对于每个输入样本 \( x \)（一个 \(\text{num\_inputs}\)-维向量），线性层的计算过程是：
     $$ y = x \cdot W + b $$
   - 这里，\( x \cdot W \) 表示输入向量与权重矩阵的矩阵乘法，结果是一个标量（因为 \( W \) 的维度是 \(\text{num\_inputs} \times 1\)）。最后加上偏置 \( b \) 产生最终的输出 \( y \)。

#### 详细步骤

1. **矩阵乘法**:
   - 将输入特征 \( X \) 与权重矩阵 \( W \) 相乘，计算结果是一个 \((N, 1)\) 维的矩阵。每个样本的特征向量通过权重矩阵变换成一个标量。

2. **加上偏置**:
   - 对于每个样本的输出，偏置 \( b \) 被加到结果上。这一步调整了输出的值，使其符合数据的实际情况。

### 总结

在 `nn.Linear(num_inputs, 1)` 定义的线性层中，网络的每一层都执行线性变换，即将输入特征通过权重矩阵和偏置项进行变换。这种变换的数学表示是：

$$ y = X \cdot W^T + b $$

这个公式表示了如何从输入特征 \( X \) 通过线性变换得到输出 \( y \)。

解释

Sequential:

Sequential 是一个容器，按顺序包含了一系列的层（modules）。在这个例子中，它只包含一个线性层。
(0): Linear(in_features=2, out_features=1, bias=True):

(0) 表示这是 Sequential 容器中的第一个（也是唯一一个）层。层的编号是从 0 开始的。
Linear(in_features=2, out_features=1, bias=True) 描述了一个线性层，其具体参数如下：
in_features=2: 输入特征的维度是 2。这意味着每个输入样本有 2 个特征。
out_features=1: 输出特征的维度是 1。这个线性层将 2 个输入特征映射到 1 个输出特征。
bias=True: 表示这个线性层包含偏置项。偏置项是一个额外的参数，用于调整输出。

# 初始化模型参数

In [ ]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

详细说明

net[0].weight.data.normal_(0, 0.01):

作用: 用正态分布（均值为 0，标准差为 0.01）初始化第一个线性层的权重。
解释:
net[0] 指的是 Sequential 容器中的第一个层，这里是 nn.Linear。
weight.data 访问层的权重数据。
normal_(0, 0.01) 是一个原地操作，用于将权重数据初始化为均值为 0，标准差为 0.01 的正态分布。


net[0].bias.data.fill_(0):

作用: 将第一个线性层的偏置初始化为 0。
解释:
bias.data 访问层的偏置数据。
fill_(0) 是一个原地操作，将所有偏置值设置为 0。

# 定义损失函数

In [ ]:
loss = nn.MSELoss()

`nn.MSELoss` 是 PyTorch 中的一个损失函数，用于计算均方误差（Mean Squared Error，MSE）。均方误差是回归任务中常用的损失函数，它用于衡量预测值与真实值之间的差距。

### 详细解释

#### 1. **均方误差（MSE）**

均方误差的数学公式是：

$$
\text{MSE}(y_{\text{hat}}, y) = \frac{1}{N} \sum_{i=1}^N (y_{\text{hat}, i} - y_i)^2
$$

其中：
- \( y_{\text{hat}} \) 是模型的预测值。
- \( y \) 是真实值。
- \( N \) 是样本数量。


# 定义优化算法

In [ ]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

这行代码创建了一个随机梯度下降（SGD）优化器，用于训练 PyTorch 模型。以下是对这行代码的详细解释：

### 代码解释

```python
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
```

### 详细说明

1. **`torch.optim.SGD`**:
   - `torch.optim.SGD` 是 PyTorch 中实现随机梯度下降（SGD）优化算法的类。SGD 是一种常用的优化算法，用于通过迭代更新模型参数以最小化损失函数。

2. **`net.parameters()`**:
   - `net` 是一个 PyTorch 模型实例（例如，`nn.Sequential` 或自定义的 `nn.Module` 类）。
   - `net.parameters()` 返回模型的所有可训练参数（权重和偏置）。这些参数是 SGD 优化器要更新的对象。

3. **`lr=0.03`**:
   - `lr` 是学习率（learning rate），它控制了每次参数更新的步长大小。在这个例子中，学习率设为 0.03。学习率是优化算法中的一个重要超参数，影响训练的收敛速度和稳定性。

### 随机梯度下降（SGD）

**随机梯度下降** 是一种优化算法，用于更新模型的权重。其基本思想是通过计算损失函数的梯度，并按梯度方向更新参数。其更新规则为：

$$
\theta = \theta - \eta \cdot \nabla_\theta J(\theta)
$$

其中：
- \(\theta\) 是模型的参数（例如权重和偏置）。
- \(\eta\) 是学习率（learning rate）。
- \(\nabla_\theta J(\theta)\) 是损失函数 \(J\) 相对于参数 \(\theta\) 的梯度。

### 总结

- **`torch.optim.SGD`** 是一个实现随机梯度下降的优化器类。
- **`net.parameters()`** 提供了需要优化的模型参数。
- **`lr=0.03`** 设置了优化器的学习率，控制了参数更新的步长。
- 在训练过程中，优化器通过迭代更新模型参数，以最小化损失函数。

# 训练

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        y = y.view(-1, 1)  # 调整标签形状
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels.view(-1, 1))  # 调整标签形状
    print(f'epoch {epoch + 1}, loss {l.item():f}')

In [ ]:
w = net[0].weight.data
# 转换 true_w 为 Tensor
true_w_tensor = torch.tensor(true_w, dtype=torch.float)
print('w的估计误差：', true_w_tensor - w.view(-1))
b = net[0].bias.data
true_b_tensor = torch.tensor(true_b, dtype=torch.float)
print('b的估计误差：', true_b_tensor - b)